In [2]:
import  torch
import  torch.nn as nn
import  torch.nn.functional as F
import  torch.optim as optim
from torchvision import datasets, transforms

from visdom import Visdom

batch_size=200
learning_rate=0.01
epochs=10

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       # transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
        transforms.ToTensor(),
        # transforms.Normalize((0.1307,), (0.3081,))
    ])),
    batch_size=batch_size, shuffle=True)



class MLP(nn.Module):

    def __init__(self):
        super(MLP, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(784, 200),
            nn.LeakyReLU(inplace=True),
            nn.Linear(200, 200),
            nn.LeakyReLU(inplace=True),
            nn.Linear(200, 10),
            nn.LeakyReLU(inplace=True),
        )

    def forward(self, x):
        x = self.model(x)
        return x


net = MLP()
optimizer = optim.SGD(net.parameters(), lr=learning_rate)
criteon = nn.CrossEntropyLoss()

viz = Visdom()

viz.line([0.], [0.], win='train_loss', opts=dict(title='train loss'))
viz.line([[0.0, 0.0]], [0.], win='test', opts=dict(title='test loss&acc.',
                                                   legend=['loss', 'acc.']))
global_step = 0

for epoch in range(epochs):

    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.view(-1, 28*28)
        logits = net(data)
        loss = criteon(logits, target)

        optimizer.zero_grad()
        loss.backward()
        # print(w1.grad.norm(), w2.grad.norm())
        optimizer.step()

        global_step += 1
        viz.line([loss.item()], [global_step], win='train_loss', update='append')

        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.item()))


    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data = data.view(-1, 28 * 28)
        logits = net(data)
        test_loss += criteon(logits, target).item()

        pred = logits.argmax(dim=1)
        correct += pred.eq(target).float().sum().item()

    viz.line([[test_loss, correct / len(test_loader.dataset)]],
             [global_step], win='test', update='append')
    viz.images(data.view(-1, 1, 28, 28), win='x')
    viz.text(str(pred.detach().cpu().numpy()), win='pred',
             opts=dict(title='pred'))

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


Train Epoch: 0 [0/60000 (0%)]	Loss: 2.302362
Train Epoch: 0 [20000/60000 (33%)]	Loss: 2.290176
Train Epoch: 0 [40000/60000 (67%)]	Loss: 2.249873

Test set: Average loss: 0.0110, Accuracy: 4372.0/10000 (44%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.188417
Train Epoch: 1 [20000/60000 (33%)]	Loss: 2.109706
Train Epoch: 1 [40000/60000 (67%)]	Loss: 2.010850

Test set: Average loss: 0.0090, Accuracy: 5379.0/10000 (54%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 1.856691
Train Epoch: 2 [20000/60000 (33%)]	Loss: 1.514214
Train Epoch: 2 [40000/60000 (67%)]	Loss: 1.369516

Test set: Average loss: 0.0064, Accuracy: 7101.0/10000 (71%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 1.196719
Train Epoch: 3 [20000/60000 (33%)]	Loss: 1.098278
Train Epoch: 3 [40000/60000 (67%)]	Loss: 1.031316

Test set: Average loss: 0.0051, Accuracy: 7265.0/10000 (73%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 1.087701
Train Epoch: 4 [20000/60000 (33%)]	Loss: 1.066496
Train Epoch: 4 [40000/60000 (67%)]	Loss: 0.933143

Test set: Average loss